In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
import scriptFile as sf
import importlib as impL
import helperFuncs as funcH
import projRelatedHelperFuncs as prHF
import os
import numpy as np
import matplotlib.pyplot as plt
os.getcwd()

In [ ]:
import torch
import os
import helperFuncs as funcH
from pandas import DataFrame as pd_df
from sklearn.metrics import confusion_matrix
impL.reload(prHF)

figsize=(8,3) 
dpi=360

cf_int = 532
impL.reload(funcH)
experiments_folder = funcH.directory_find('cf{:d}'.format(cf_int), root=funcH.getVariableByComputerName('data_dir'))
print(experiments_folder)

# update_centroid_df
# analyze_correspondance_results
# map_predictions
# _cumsum_preds - prHF.cumsum_preds
# calc_tup_sc - prHF.calc_tup_sc

In [ ]:
impL.reload(prHF)
def get_epoch(ep_str):
    print(sil_var[ep_str].keys())
    silhouette_values = sil_var[ep_str]["silhouette_values"]
    silhouette_avg = sil_var[ep_str]["silhouette_avg"]
    reconstruction_loss = sil_var[ep_str]["reconstruction_loss"]
    correspondance_weights = sil_var[ep_str]["correspondance_weights"]
    sil_var[ep_str]["predictions"] = np.asarray(sil_var[ep_str]["predictions"],dtype=int)
    labels = sil_var[ep_str]["labels"]
    correspondance_tuple = sil_var[ep_str]["correspondance_tuple"]
    kluster_centroids = sil_var[ep_str]["kluster_centroids"]
    return sil_var[ep_str]

exp_fold = experiments_folder
silhouette_reconstruction_sort_file = os.path.join(exp_fold, "silhouette_reconstruction.torch")
sil_var = torch.load(silhouette_reconstruction_sort_file, map_location=torch.device('cpu'))
print(len(sil_var))
print(sil_var.keys())
for ep_str in sil_var.keys():
    _evs = get_epoch(ep_str=ep_str)
    epoch_id = int(ep_str.replace("epoch", ""))
    prHF.calc_tup_sc(_evs["silhouette_values"], _evs["reconstruction_loss"], 
                     _evs["correspondance_tuple"], _evs["labels"], 
                     ep_id=epoch_id, experiments_folder=exp_fold)

In [ ]:
kluster_centroids = prHF.update_centroid_df(_evs["kluster_centroids"], _evs["predictions"])
prHF.analyze_correspondance_results(_evs["correspondance_tuple"], _evs["kluster_centroids"], _evs["predictions"], _evs["labels"])

In [ ]:
result_dict = funcH.analyze_silhouette_values(_evs["silhouette_values"], _evs["predictions"], _evs["labels"],
                                    centroid_info_pdf=_evs["kluster_centroids"],
                                    label_names=None, conf_plot_save_to='',
                                    figsize=figsize, lw=[4, 3, 2], show_title=False, str_deg=15, str_size=12)
sil_val_sorted, sil_idx = funcH.sortVec(_evs["silhouette_values"])
rec_los_sorted, rec_idx = funcH.sortVec(-_evs["reconstruction_loss"])
rec_los_sorted_0_1 = funcH.map_0_1(-rec_los_sorted)
predictions_mapped = prHF.map_predictions(_evs["labels"], _evs["predictions"], centroid_info_pdf=_evs["kluster_centroids"])
print(np.min(rec_los_sorted), np.max(rec_los_sorted))

In [ ]:
cumsum_preds_sil = prHF.cumsum_preds(labels, predictions_mapped, sil_idx)
cumsum_preds_rec = prHF.cumsum_preds(labels, predictions_mapped, rec_idx)
data_perc_vec = np.arange(0, len(cumsum_preds_rec)) / len(cumsum_preds_rec)

plt.close('all')
fig, ax = plt.subplots(1, figsize=figsize, dpi=dpi)
ax.plot(data_perc_vec, cumsum_preds_sil, lw=2, label='sil_cs', color='blue', ls='-', zorder=0)
ax.plot(data_perc_vec, cumsum_preds_rec, lw=2, label='rec_cs', color='red', ls='-', zorder=0)
ax.plot(data_perc_vec, sil_val_sorted, lw=1, label='sil_val_sorted', color='cyan', ls='-', zorder=0)
ax.plot(data_perc_vec, rec_los_sorted_0_1, lw=1, label='rec_los_sorted_0_1', color='orange', ls='-', zorder=0)
plt.legend(loc='upper right')
np.mean(rec_los_sorted_0_1)